<a href="https://colab.research.google.com/github/Nahgalvao/Trabalho_Reologia_UNESP/blob/main/Reologia_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este pacote é um conjunto de funções que primeiro analisa os dados do Anton-Parr
rheometer em uma série de arrays numpy e os salva (rheologyDataParserOld).

Depois que os dados são salvos em uma série de matrizes numpy, todas as matrizes podem ser calculadas juntas
e salvo em uma matriz numpy média usando dataSetPackager.

Todos os dados

In [ ]:
import numpy as np
import os
import fittingFunctions

ModuleNotFoundError: ignored

In [ ]:
def rheologyDataParserOld( fileName, outputDirectory ):

Esta função pega os dados do formato antigo do reômetro Anton-Parr e os analisa em uma série de arquivos de texto que tem menos informações, mas também torna os dados mais fáceis de lidar. Para obter um sentimento para o que esta função está fazendo, dê uma olhada no formato antigo e veja como ele está organizado primeiro.
    
O fileName é o fileName do arquivo de texto que foi gerado pelo programa Anton-Parr que está no formato que é obtido quando se faz arquivo-->exportar-->conteúdo da janela ativa-->arquivo de texto separado por tabulações. O arquivo exportado devem ser todos da mesma fração de embalagem. O diretório de saída é onde cada conjunto de dados é gerado como um numpy arquivo salvo. Cada arquivo numpy possui uma matriz [N,6] onde N é o número de pontos de dados obtidos e os 6 correspondem as 6 quantidades medidas durante um experimento de taxa de cisalhamento constante [ponto #, tempo, viscosidade, taxa de cisalhamento, tensão de cisalhamento, torque].

* Carregando o arquivo a ser analisado.
* Lemos o arquivo inteiro como uma lista de linhas para facilitar a análise.





In [ ]:
    with open(fileName, "rb") as f1:
         lines = f1.readlines()

* Precisamos percorrer e encontrar todas as linhas que têm "Data Series Information\n" neles porque essas linhas são as que segmentam nossos dados em diferentes experimentos.

In [ ]:
    linesWithDataSeriesInfo = np.zeros([0])
    counter = 0
    for line in lines:
        if "Data Series Information\r\n" in line.decode(encoding="utf-8", errors='ignore'):
            linesWithDataSeriesInfo= np.append(linesWithDataSeriesInfo , counter)
        counter = counter + 1

* Agora sabemos que o número total de conjuntos de dados que temos é:

In [ ]:
    totalNumOfExperiments = len(linesWithDataSeriesInfo)

    for i in range(0,totalNumOfExperiments):

* Isso seleciona um dos blocos de dados e agora precisamos analisá-lo em experimentos separados e o nome do experimento. Estes entrarão em um arquivo de texto que está em um diretório com o nome do experimento

In [ ]:
        if i == totalNumOfExperiments-1:
            currentDataSet = lines[ int(linesWithDataSeriesInfo[i]) : int(len(lines)) ]
        else:
            currentDataSet = lines[ int(linesWithDataSeriesInfo[i]) : int(linesWithDataSeriesInfo[i+1]) ]

* Agora precisamos escolher esse conjunto de dados e obter todas as informações que queremos dele.
* Primeiro, obteremos o nome do experimento percorrendo todas as linhas do conjunto de dados e encontrar o nome.

In [ ]:
        for j in range(0, len(currentDataSet)):
            if "Name:" in currentDataSet[j].decode(encoding="utf-8", errors='ignore'):
                nameOfCurrentDataSet = currentDataSet[j][8:]

        nameOfCurrentDataSet = nameOfCurrentDataSet.decode(encoding="utf-8", errors='ignore').replace("\r\n","")

* Agora que temos o nome do conjunto de dados podemos criar um novo diretório dentro do diretório de saída com o nome do experimento.

In [ ]:
        filePath = (outputDirectory+os.sep+nameOfCurrentDataSet)
        os.mkdir(filePath)

* Agora queremos percorrer e encontrar todas as linhas em nosso conjunto de dados que possuem:
* "\t[s]\t[Pa·s]\t[1/s]\t[Pa]\t[mNm]\t[]\n"
  porque esse é o começo de um conjunto de dados.

In [ ]:
        linesWhereDataStarts = np.zeros([0])
        for k in range(0, len(currentDataSet)):
            if "\t[s]\t[Pas]\t[1/s]\t[Pa]\t[mNm]\t[]\r\n" in currentDataSet[k].decode(encoding="utf-8", errors='ignore'):
                linesWhereDataStarts = np.append(linesWhereDataStarts, k)


        linesWhereDataStarts=[int(i) for i in linesWhereDataStarts]

* Agora que sabemos onde os dados começam, podemos começar a selecionar conjuntos de dados individuais

In [ ]:
        linesWhereDataEnds = np.zeros([0])
        for j in linesWhereDataStarts:
            for k in range(j,len(currentDataSet)):
                if currentDataSet[k].decode(encoding="utf-8", errors='ignore') == "\r\n":
                    linesWhereDataEnds = np.append(linesWhereDataEnds, k)
                    break

        linesWhereDataEnds=[int(i) for i in linesWhereDataEnds]

        if len(linesWhereDataEnds)!=len(linesWhereDataStarts):
            linesWhereDataEnds = np.append(linesWhereDataEnds, len(currentDataSet) )


        for j in range(0,len(linesWhereDataEnds)):

            filePathFinal = (outputDirectory+os.sep+nameOfCurrentDataSet)
            dataSetToSave = currentDataSet[linesWhereDataStarts[j]+1:linesWhereDataEnds[j]]
            finalDataSet = np.zeros((len(dataSetToSave),6))

            for k in range(0,len(dataSetToSave)):
                finalDataSet[k,:]=np.fromstring(dataSetToSave[k].decode(encoding="utf-8", errors='ignore').replace('\t',' ').replace(',', '').replace('\n',''),sep=' ')
            finalNameForData = r"dataSet-" + str(j) + r".txt"
            filePathFinal = (outputDirectory+os.sep+nameOfCurrentDataSet+os.sep+finalNameForData)
            np.save(filePathFinal,finalDataSet)


def dataSetPackager(topDir,baseViscosity=False):

* Esta função pega o caminho topDir e encontra todos os arquivos de dados que terminam em .npy, esses arquivos devem ter sido gerados pela função acima "oldRheologyDataParser". Essa função pega os dados brutos do reômetro e os empacota em uma matriz que possui linhas organizadas como [pt#, tempo, viscosidade, taxa de cisalhamento, tensão de cisalhamento, torque]. Esta função carrega em cada curva de fluxo individual gerada por "oldRheologyDataParser" e calcula a média dos valores x (tensão de cisalhamento) e calcula a média dos valores y (viscosidade) e gera erros para o ponto de dados médio.
    
*VOCÊ DEVE EXCLUIR SEUS ARQUIVOS .npy PRESHEAR DE SEU DIRETÓRIO topDir OU ENTÃO ESTA FUNÇÃO IRÁ TENTAR FAZER A MÉDIA DAS entradas:
* topDir é o diretório no qual todas as curvas de fluxo que você gostaria de calcular juntas. Exclua os pré-shears desses diretórios CERTIFIQUE-SE DE COLOCAR r ANTES DA STRING, POR EXEMPLO topDir = r"blah"
* saídas:
* It

* Inicialize os contêineres que conterão todos os conjuntos de dados.

In [ ]:
    dataSetHolder = np.array([])

* Este loop encontrará todos os arquivos no topDir que terminam com .npy

In [ ]:
    for dirpath, dirnames, filenames in os.walk(topDir):
        for filename in [f for f in filenames if f.endswith(r".npy")]:

            if (filename == r"averagedData.npy") or (filename == r"errorsData.npy"):
              break

* Este é o conjunto de dados atual organizado como [pt#, tempo, viscosidade, taxa de cisalhamento, tensão de cisalhamento, torque]

In [ ]:
            currentDataSet = np.expand_dims(np.load(os.path.join(dirpath, filename)),axis=2)

* O conjunto de dados atual deve ser ordenado da taxa de cisalhamento controlada mais baixa para a taxa de cisalhamento mais alta, então verificamos e invertemos se necessário.

In [ ]:
            if currentDataSet[0,4,0] > currentDataSet[-1,4,0]:
                currentDataSet = np.flip(currentDataSet,axis=0)
            print(dirpath+os.sep+filename)
            if dataSetHolder.any():
                dataSetHolder = np.append(dataSetHolder , currentDataSet , axis=2 )
            else:
                dataSetHolder = currentDataSet


    if baseViscosity != False:
        dataSetHolder[:,2,:] = dataSetHolder[:,2,:]/baseViscosity

    averagedData = np.mean(dataSetHolder,axis=2)
    standardDeviationData = np.std(dataSetHolder,axis=2)

    if baseViscosity == False:
        np.save(os.path.join(topDir, r"averagedData"), averagedData )
    else:
        np.save(os.path.join(topDir, r"averagedDataVR"), averagedData )


    if baseViscosity == False:
        np.save(os.path.join(topDir, r"errorsData"), standardDeviationData )
    else:
        np.save(os.path.join(topDir, r"errorsDataVR"), standardDeviationData )

    return (averagedData,standardDeviationData)


def dataSetCollector(topDir,whichDataSets,finalFileName,flipOrNot=False):

    dataSetToJoin = []

    for i in whichDataSets:


        if dataSetToJoin == []:
            dataSetToJoin = np.load(os.path.join(topDir, r"dataSet-" +str(i)+r".txt.npy" ))
        else:
            dataSetToJoin = np.concatenate((dataSetToJoin,np.load(os.path.join(topDir, r"dataSet-" +str(i)+r".txt.npy" ))),axis=0)

    if flipOrNot ==  True:
        dataSetToJoin = np.flip(dataSetToJoin,axis=0)

    np.save(os.path.join(topDir,finalFileName),dataSetToJoin)



def tauStarDataPackager(topDir,rescaledViscosity=False):

    packagedData = []

    listOfSubDirs = next(os.walk(topDir))[1]

    for packingFraction in listOfSubDirs:

        if rescaledViscosity ==False:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedData.npy" ))
        else:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedDataVR.npy" ))

        (numRows,_) = np.shape(loadInAveragedData)

        if packagedData == []:
            packagedData = np.concatenate((float(packingFraction)*np.ones((numRows,1)),np.expand_dims(loadInAveragedData[:,4],axis=1),np.expand_dims(loadInAveragedData[:,3],axis=1)),axis=1)
        else:
            packagedData = np.concatenate((packagedData,np.concatenate((float(packingFraction)*np.ones((numRows,1)),np.expand_dims(loadInAveragedData[:,4],axis=1),np.expand_dims(loadInAveragedData[:,3],axis=1)),axis=1)),axis=0)
    np.save(os.path.join(topDir,"tauStarPackagedData"),packagedData)
    return packagedData



def prepDataPackager(topDir,errorCornstarchMass,errorSolventMass,errorSolventRho):


    errorData = []
    listOfSubDirs = next(os.walk(topDir))[1]

    for packingFraction in listOfSubDirs:

        currentData = np.loadtxt(os.path.join(topDir, packingFraction , r"prepDetails" ))

        currentData = np.append(currentData,float(packingFraction)/100)

        currentData = np.transpose(np.expand_dims(currentData,axis=1))

        soluteMass = currentData[0,1]
        solventMass = currentData[0,1]
        solventRho = currentData[0,2]
        soluteRho = currentData[0,3]
        errorInCurrentPhi = fittingFunctions.errorInPhiDataPoint(soluteMass,solventMass,soluteRho,solventRho,errorCornstarchMass,errorSolventMass,errorSolventRho)
        errorInCurrentPhi = np.expand_dims(errorInCurrentPhi,axis=0)
        np.save(os.path.join(topDir, packingFraction , r"errorInPhi" ),errorInCurrentPhi)

        if errorData == []:
            errorData = errorInCurrentPhi
        else:
            errorData = np.concatenate((errorData,errorInCurrentPhi),axis=0)



    return errorData


def convertNicolesData(topDir):


    listOfSubDirs = next(os.walk(topDir))[1]

    for packingFraction in listOfSubDirs:



        onlyfiles = [f for f in os.listdir(os.path.join(topDir,packingFraction)) if os.path.isfile(os.path.join(os.path.join(topDir,packingFraction), f))]

        nicolesData = np.loadtxt(os.path.join(topDir,packingFraction,onlyfiles[0]))

        (numRows,_) = np.shape(nicolesData)

        averagedData = np.zeros((numRows,6))
        errorData = np.zeros((numRows,6))

        averagedData[:,2] = nicolesData[:,2]
        averagedData[:,3] = nicolesData[:,1]
        averagedData[:,4] = nicolesData[:,0]
        errorData[:,2] = nicolesData[:,3]


        np.save(os.path.join(topDir,packingFraction,"averagedData"),averagedData)
        np.save(os.path.join(topDir,packingFraction,"errorsData"),errorData)

####

In [ ]:
from scipy.optimize import curve_fit
import numpy as np
import os
import plottingFunctions

def tauStarFit(packingFractionValues,stressValues,shearRateValues,phi0,phiM,alpha,suspendingViscosity):

* X é uma tupla que é X = (packingFractionValues,stressValues)

In [ ]:
    def strainRateFunc(X,tauStar):
        packingFraction,shearStress = X

        return (1/suspendingViscosity)*shearStress*(1- packingFraction/(phiM+(phi0-phiM)*np.exp(-shearStress/tauStar)))**alpha

    (popt, pcov) = curve_fit(strainRateFunc, (packingFractionValues,stressValues), shearRateValues,bounds=(0, [10000]))

    return popt,pcov


def allParameterFit(packingFractionValues,stressValues,shearRateValues,suspendingViscosity):

* X é uma tupla que é X = (packingFractionValues,stressValues)

In [ ]:
    def strainRateFunc(X,tauStar,phiM,phi0,alpha):
        packingFraction,shearStress = X

        return (1/suspendingViscosity)*shearStress*(1- packingFraction/(phiM+(phi0-phiM)*np.exp(-shearStress/tauStar)))**alpha

    (popt, pcov) = curve_fit(strainRateFunc, (packingFractionValues,stressValues), shearRateValues,bounds=(0, [100,.8,.8,5]))

    return popt,np.sqrt(np.diag(pcov))


def tauStarFitStretchExponential(packingFractionValues,stressValues,shearRateValues,phi0,phiM,alpha,suspendingViscosity):

* X é uma tupla que é X = (packingFractionValues,stressValues)

In [ ]:
    def strainRateFunc(X,tauStar,beta):
        packingFraction,shearStress = X

        return (1/suspendingViscosity)*shearStress*(1- packingFraction/(phiM+(phi0-phiM)*np.exp((-shearStress/tauStar)**beta)))**alpha

    (popt, pcov) = curve_fit(strainRateFunc, (packingFractionValues,stressValues), shearRateValues,bounds=([0,.999], [10000,1]))

    return popt,pcov

def errorInPhiDataPoint(cornstarchMass,solventMass,cornstarchRho,solventRho,errorCornstarchMass,errorSolventMass,errorSolventRho):


    densityDenom = (solventMass/solventRho) + (cornstarchMass/cornstarchRho)

    dPhi_dCornstarchMass = 1/(cornstarchRho*densityDenom) - cornstarchMass/(cornstarchRho*densityDenom)**2

    dPhi_dSolventMass = -cornstarchMass/(cornstarchRho*solventRho*densityDenom**2)

    dPhi_dSolventDensity = cornstarchMass*solventMass/(cornstarchRho*(solventRho*densityDenom)**2)

    return np.sqrt((dPhi_dCornstarchMass*errorCornstarchMass)**2 + (dPhi_dSolventMass*errorSolventMass)**2 + (dPhi_dSolventDensity*errorSolventRho)**2)


def newtonianPlateauFinder(topDir,minViscosityList,maxViscosityList,rescaleViscosity=False,tauMin=False,tauMax=False):

    listOfSubDirs = next(os.walk(topDir))[1]
    listOfSubDirs = [float(i) for i in listOfSubDirs]
    listOfSubDirs.sort()
    listOfSubDirs = [str(i) for i in listOfSubDirs]
    listOfSubDirs = listOfSubDirs[::-1]

    if (len(minViscosityList)!=len(listOfSubDirs)) or (len(maxViscosityList)!=len(listOfSubDirs)):
        return print("The length of minViscosityList and the length of maxViscosityList must be the same as the number of packing fractions in topDir.")

    phiVsMaxViscosity = np.array([])
    phiVsMinViscosity = np.array([])

    counter = 0
    for packingFraction in listOfSubDirs:

        if rescaleViscosity == False:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedData.npy" ))
            loadInErrorData = np.load(os.path.join(topDir, packingFraction , r"errorsData.npy" ))
        else:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedDataVR.npy" ))
            loadInErrorData = np.load(os.path.join(topDir, packingFraction , r"errorsDataVR.npy" ))

        sortedViscosities = np.sort(loadInAveragedData[:,2])

        if minViscosityList[counter] == -1:
            minViscosity = np.nan
            errorOnMinViscosity = np.nan
        else:
            if tauMin == False:
                minViscosity = sortedViscosities[minViscosityList[counter]]
                indexOfMinViscosity = np.where(loadInAveragedData[:,2]==minViscosity)
                errorOnMinViscosity = loadInErrorData[indexOfMinViscosity[0][0],2]
                stressOfMinViscosity = loadInAveragedData[indexOfMinViscosity,4]
            else:
                tauMinOnset = plottingFunctions.find_nearest(loadInAveragedData[:,4],tauMin)
                minViscosity = loadInAveragedData[np.where(loadInAveragedData[:,4]==tauMinOnset),2][0]
                indexOfMinViscosity = np.where(loadInAveragedData[:,2]==minViscosity)
                errorOnMinViscosity = loadInErrorData[indexOfMinViscosity[0][0],2]
                indexOfMinViscosity = np.where(loadInAveragedData[:,2]==minViscosity)
                stressOfMinViscosity = loadInAveragedData[indexOfMinViscosity,4]


        if maxViscosityList[counter] == -1:
            maxViscosity = np.nan
            errorOnMaxViscosity = np.nan
        else:
            if tauMax == False:

                maxViscosity = sortedViscosities[-(maxViscosityList[counter]+1)]
                indexOfMaxViscosity = np.where(loadInAveragedData[:,2]==maxViscosity)
                errorOnMaxViscosity = loadInErrorData[indexOfMaxViscosity[0][0],2]
                stressOfMaxViscosity = loadInAveragedData[indexOfMaxViscosity,4]

            else:
                tauMaxOnset = plottingFunctions.find_nearest(loadInAveragedData[:,4],tauMax)
                maxViscosity = loadInAveragedData[np.where(loadInAveragedData[:,4]==tauMaxOnset),2][0]
                if len(maxViscosity)!=1:
                    maxViscosity=maxViscosity[0]
                indexOfMaxViscosity = np.where(loadInAveragedData[:,2]==maxViscosity)
                errorOnMaxViscosity = loadInErrorData[indexOfMaxViscosity,2]
                indexOfMaxViscosity = np.where(loadInAveragedData[:,2]==maxViscosity)
                stressOfMaxViscosity = loadInAveragedData[indexOfMaxViscosity,4]


        counter = counter +1
        if phiVsMinViscosity.size == 0:

            phiVsMinViscosity = np.array([float(packingFraction)/100 ,minViscosity,stressOfMinViscosity])
            phiVsMaxViscosity = np.array([float(packingFraction)/100 ,maxViscosity,stressOfMaxViscosity])
            phiVsMinViscosityErrors = np.array([float(packingFraction)/100 ,errorOnMinViscosity])
            phiVsMaxViscosityErrors = np.array([float(packingFraction)/100 ,errorOnMaxViscosity])

        else:
            if tauMin==False:
                phiVsMinViscosity = np.vstack( (phiVsMinViscosity,np.array([float(packingFraction)/100 ,minViscosity,stressOfMinViscosity])))
            else:
                phiVsMinViscosity = np.vstack( (phiVsMinViscosity,np.array([float(packingFraction)/100 ,minViscosity[0],stressOfMinViscosity[0]])))

            phiVsMaxViscosity = np.vstack( (phiVsMaxViscosity,np.array([float(packingFraction)/100 ,maxViscosity,stressOfMaxViscosity])) )
            phiVsMinViscosityErrors = np.vstack( (phiVsMinViscosityErrors,np.array([float(packingFraction)/100 ,errorOnMinViscosity])))
            phiVsMaxViscosityErrors = np.vstack( (phiVsMaxViscosityErrors,np.array([float(packingFraction)/100 ,errorOnMaxViscosity])))


    phiVsMinViscosity = phiVsMinViscosity[phiVsMinViscosity[:,0].argsort()]
    phiVsMaxViscosity = phiVsMaxViscosity[phiVsMaxViscosity[:,0].argsort()]
    phiVsMaxViscosityErrors = phiVsMaxViscosityErrors[phiVsMaxViscosityErrors[:,0].argsort()]
    phiVsMinViscosityErrors = phiVsMinViscosityErrors[phiVsMinViscosityErrors[:,0].argsort()]

    phiVsMinViscosity = phiVsMinViscosity[~np.isnan(phiVsMinViscosity).any(axis=1)]
    phiVsMaxViscosity = phiVsMaxViscosity[~np.isnan(phiVsMaxViscosity).any(axis=1)]
    phiVsMinViscosityErrors = phiVsMinViscosityErrors[~np.isnan(phiVsMinViscosityErrors).any(axis=1)]
    phiVsMaxViscosityErrors = phiVsMaxViscosityErrors[~np.isnan(phiVsMaxViscosityErrors).any(axis=1)]




    return (phiVsMinViscosity,phiVsMaxViscosity,phiVsMinViscosityErrors,phiVsMaxViscosityErrors)


def viscosityDivergenceFit(packingFractions,viscosities,viscosityErrors,suspendingViscosity=False,fixAlpha=False):

    if (suspendingViscosity!=False) and (fixAlpha!=False):
        def divergingViscosityFunc(x,phiJ):
            return  suspendingViscosity*( 1 - (x/phiJ) )**( -fixAlpha )

        (phiJ, phiJError) =curve_fit(divergingViscosityFunc,packingFractions,viscosities,sigma=viscosityErrors)
        return (phiJ, phiJError)

    if (suspendingViscosity!=False) and (fixAlpha==False):
        def divergingViscosityFunc(x,phiJ,alpha):
            return  suspendingViscosity*( 1 - (x/phiJ) )**( -alpha )
        (popt, pcov) =curve_fit(divergingViscosityFunc,packingFractions,viscosities,sigma=viscosityErrors)
        errors = np.sqrt(np.diag(pcov))
        phiJ = popt[0]
        alpha = popt[1]
        phiJError = errors[0]
        alphaError = errors[1]
        return (phiJ, alpha,phiJError,alphaError)


    if (suspendingViscosity==False) and (fixAlpha==False):
        def divergingViscosityFunc(x,phiJ,alpha):
            return  ( 1 - (x/phiJ) )**( -alpha )
        (popt, pcov) =curve_fit(divergingViscosityFunc,packingFractions,viscosities,sigma=viscosityErrors)
        errors = np.sqrt(np.diag(pcov))
        phiJ = popt[0]
        alpha = popt[1]
        phiJError = errors[0]
        alphaError = errors[1]
        return (phiJ, alpha,phiJError,alphaError)


    if (suspendingViscosity==False) and (fixAlpha!=False):
        def divergingViscosityFunc(x,phiJ):
            return  ( 1 - (x/phiJ) )**( -fixAlpha )

        (phiJ, phiJError) =curve_fit(divergingViscosityFunc,packingFractions,viscosities,sigma=viscosityErrors)
        return (phiJ, phiJError)

####

In [ ]:
import numpy as np
import os
from matplotlib import pyplot
import fittingFunctions



def shearStressVsViscosityPlotter(topDir,titleOfPlot="normal",rescaledViscosity=False,noErrors=False):

* Obtenha uma lista de todos os subdiretórios

In [ ]:
    listOfSubDirs = next(os.walk(topDir))[1]
    listOfSubDirs = [float(i) for i in listOfSubDirs]
    listOfSubDirs.sort()
    listOfSubDirs = [str(i) for i in listOfSubDirs]
    listOfSubDirs = listOfSubDirs[::-1]

    f, ax = pyplot.subplots()


    for packingFraction in listOfSubDirs:

        if rescaledViscosity == False:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedData.npy" ))
        else:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedDataVR.npy" ))


        if noErrors == False:
            if rescaledViscosity ==False:
                loadInErrordata = np.load(os.path.join(topDir, packingFraction , r"errorsData.npy" ))
            else:
                loadInErrordata = np.load(os.path.join(topDir, packingFraction , r"errorsDataVR.npy" ))

* Primeiro a plotagem em função da tensão de cisalhamento.

In [ ]:
        if noErrors == False:
            ax.errorbar(loadInAveragedData[:,4],loadInAveragedData[:,2],xerr= loadInErrordata[:,4],yerr=loadInErrordata[:,2],label=packingFraction,marker='o',markersize=5)
        else:
            ax.plot(loadInAveragedData[:,4],loadInAveragedData[:,2],label=packingFraction,marker='o')

* Em seguida, plote a viscosidade em função da taxa de cisalhamento;

In [ ]:
    pyplot.yscale('log')
    pyplot.xscale('log')
    pyplot.xlabel("Shear Stress [Pa]")

    if rescaledViscosity==False:
        pyplot.ylabel("Viscosity [Pa s]")
    else:
        pyplot.ylabel(r"Rescaled Viscosity $\frac{\eta}{\eta_0}$")

    pyplot.title(titleOfPlot)

    pyplot.grid(True)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    return f, ax

def tauMinTauMaxPlotter(topDir,tauMin,tauMax,titleOfPlot="normal",rescaledViscosity=False,noErrors=False):


    listOfSubDirs = next(os.walk(topDir))[1]

    minViscosityList = np.zeros(len(listOfSubDirs))
    maxViscosityList = np.zeros(len(listOfSubDirs))

    (phiVsMinViscosity,phiVsMaxViscosity,phiVsMinViscosityErrors,phiVsMaxViscosityErrors) = fittingFunctions.newtonianPlateauFinder(topDir,minViscosityList,maxViscosityList,rescaledViscosity,tauMin,tauMax)

    f, ax = shearStressVsViscosityPlotter(topDir,titleOfPlot,rescaledViscosity,noErrors)

    (numSystems,_) = np.shape(phiVsMaxViscosity)

    for i in range(0,numSystems):

        ax.plot(phiVsMinViscosity[i,:][2],phiVsMinViscosity[i,:][1],'o',markerSize=15,color="red")
        ax.plot(phiVsMaxViscosity[i,:][2],phiVsMaxViscosity[i,:][1],'o',markerSize=15,color="blue")

    return f,ax




def shearRateVsViscosityPlotter(topDir,solvent="normal",rescaleViscosity=False):

* Obtenha uma lista de todos os subdiretórios

In [ ]:
    listOfSubDirs = next(os.walk(topDir))[1]
    listOfSubDirs = [float(i) for i in listOfSubDirs]
    listOfSubDirs.sort()
    listOfSubDirs = [str(i) for i in listOfSubDirs]
    listOfSubDirs = listOfSubDirs[::-1]


    ax = pyplot.subplot(111)

    for packingFraction in listOfSubDirs:

        if rescaleViscosity ==False:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedData.npy" ))
            loadInErrordata = np.load(os.path.join(topDir, packingFraction , r"errorsData.npy" ))
        else:
            loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedDataVR.npy" ))
            loadInErrordata = np.load(os.path.join(topDir, packingFraction , r"errorsDataVR.npy" ))

NameError: ignored

* Primeiro a plotagem em função da tensão de cisalhamento.

In [ ]:
        ax.errorbar(loadInAveragedData[:,3],loadInAveragedData[:,2],xerr= loadInErrordata[:,3],yerr=loadInErrordata[:,2],label=packingFraction,marker='o')

* Em seguida, plote a viscosidade em função da taxa de cisalhamento,

In [ ]:
    pyplot.yscale('log')
    pyplot.xscale('log')
    pyplot.xlabel(r"Shear Rate [$\frac{1}{s}$]")

    if rescaleViscosity==False:
        pyplot.ylabel("Viscosity [Pa s]")
    else:
        pyplot.ylabel(r"Rescaled Viscosity $\frac{\eta}{\eta_0}$")

    if solvent == "GdCl":
        pyplot.title("Flow Curves for GdCl Cornstarch Suspensions")
    if solvent =="normal":
        pyplot.title("Flow Curves for Normal Cornstarch Suspensions")

    pyplot.grid(True)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))




def viscosityDivergencePlot(viscosityValues,packingFractionValues,viscosityErrors,labelsForCurves,fixAlpha=False,titleOfPlot ="Newtonian Plateau's Divergences"):


    shapeOfVis = np.shape(viscosityValues)

    if np.shape(shapeOfVis)[0]==1:
        numSystems=1
    else:
        numSystems = shapeOfVis[0]

    ax = pyplot.subplot(111)

    if fixAlpha== False:
        def divergingViscosityFunc(x,phiJ, alpha):
            return ( 1 - (x/phiJ) )**( -alpha )
    else:
        def divergingViscosityFunc(x,phiJ):
            return ( 1 - (x/phiJ) )**(-fixAlpha)





    phiJHolder = np.zeros(numSystems)
    phiJErrorHolder = np.zeros(numSystems)

    if fixAlpha == False:
        alphaHolder = np.zeros(numSystems)
        alphaErrorHolder = np.zeros(numSystems)


    for i in range(0,numSystems):
        print(numSystems)
        if numSystems == 1:
            currentViscosityErrors = viscosityErrors
            currentPhiValues = packingFractionValues
            currentViscosityValues = viscosityValues
        else:
            currentViscosityErrors = viscosityErrors[i,:]
            currentPhiValues = packingFractionValues[i,:]
            currentViscosityValues = viscosityValues[i,:]


        currentViscosityValues = currentViscosityValues[~np.isnan(currentViscosityValues)]
        currentViscosityErrors = currentViscosityErrors[~np.isnan(currentViscosityValues)]
        currentPhiValues = currentPhiValues[~np.isnan(currentPhiValues)]

        if fixAlpha == False:
            (phiJHolder[i],alphaHolder[i],phiJErrorHolder[i],alphaErrorHolder[i]) = fittingFunctions.viscosityDivergenceFit(currentPhiValues,currentViscosityValues,currentViscosityErrors,False,False)
            ax.plot(np.linspace(0,phiJHolder[i]-.001,1000),divergingViscosityFunc(np.linspace(0,phiJHolder[i]-.001,1000),phiJHolder[i],alphaHolder[i]),label=labelsForCurves[i] +" Fit" )

            minimumError = divergingViscosityFunc(np.linspace(0,phiJHolder[i]-.001,1000),phiJHolder[i]-phiJErrorHolder[i],alphaHolder[i]+alphaErrorHolder[i])
            maximumError = divergingViscosityFunc(np.linspace(0,phiJHolder[i]-.001,1000),phiJHolder[i]+phiJErrorHolder[i],alphaHolder[i]-alphaErrorHolder[i])
            #pyplot.plot(np.linspace(0,phiJHolder[i]-.001,1000),minimumError,label="minimumError")
            #pyplot.plot(np.linspace(0,phiJHolder[i]-.001,1000), maximumError,label="maximumError")

            ax.fill_between(np.linspace(0,phiJHolder[i]-.001,1000), minimumError, maximumError, where=maximumError <= minimumError, facecolor='lightblue')
        else:
            (phiJHolder[i],phiJErrorHolder[i]) = fittingFunctions.viscosityDivergenceFit(currentPhiValues,currentViscosityValues,currentViscosityErrors,False,fixAlpha)
            minimumError = divergingViscosityFunc(np.linspace(0,phiJHolder[i]-.001,1000),phiJHolder[i]-phiJErrorHolder[i])
            maximumError = divergingViscosityFunc(np.linspace(0,phiJHolder[i]-.001,1000),phiJHolder[i]+phiJErrorHolder[i])
            ax.fill_between(np.linspace(0,phiJHolder[i]-.001,1000), minimumError, maximumError, where=maximumError <= minimumError, facecolor='lightblue')

            ax.plot(np.linspace(0,phiJHolder[i]-.001,1000),divergingViscosityFunc(np.linspace(0,phiJHolder[i]-.001,1000),phiJHolder[i]),label=labelsForCurves[i] +" Fit" )
        print(currentViscosityErrors)
        ax.errorbar(currentPhiValues, currentViscosityValues,fmt='o',yerr=currentViscosityErrors,label=labelsForCurves[i])



    ax.legend()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    pyplot.title(titleOfPlot)
    pyplot.xlabel("Packing Fraction $\phi$")
    pyplot.ylabel("Rescaled Viscosity $\eta_R$")
    pyplot.xlim(np.nanmin(packingFractionValues)-.05,np.nanmax(packingFractionValues)+.04)
    pyplot.ylim(0,np.nanmax(viscosityValues)+20)
    if fixAlpha == False:
        return (phiJHolder,alphaHolder,phiJErrorHolder,alphaErrorHolder)
    else:
        return (phiJHolder,phiJErrorHolder)


def logarithmicPlotter(viscosityValues0,viscosityValues1,packingFractionValues0,packingFractionValues1,alpha0,phiJ0,alpha1,phiJ1,highOrLow):

    logViscosityValues0 = np.log10(viscosityValues0)
    logPackingFraction0 = np.log10(1-packingFractionValues0/phiJ0)

    logViscosityValues1 = np.log10(viscosityValues1)
    logPackingFraction1 = np.log10(1-packingFractionValues1/phiJ1)


    pyplot.plot(logPackingFraction0,logViscosityValues0,'o')

    pyplot.plot(logPackingFraction1,logViscosityValues1,'o')
    #pyplot.title()
    if highOrLow == "high":
        pyplot.xlabel(r"$\log(1-\frac{\phi}{\phi_M})$")
        pyplot.title("Frictional Newtonian Regimes")

    if highOrLow == "low":
        pyplot.xlabel(r"$\log(1-\frac{\phi}{\phi_0})$")
        pyplot.title("Frictionless Newtonian Regimes")


    pyplot.legend(("Pure Cornstarch Suspension","GdCl Cornstarch Suspension"))
    pyplot.ylabel(r"$\log(\eta)$")

    font = {'family' : 'normal',

        'size'   : 10}

    pyplot.rc('font', **font)
    #pyplot.legend(loc='center left', bbox_to_anchor=(1, 0.5))


def logarithmicPlotterAllLines(phiVsMinViscosityG,phiVsMinViscosityC,phiVsMaxViscosityG,phiVsMaxViscosityC,phiMG,alphaMG,phi0G,alpha0G,phiMC,alphaMC,phi0C,alpha0C,suspendViscosityC,suspendViscosityG):


    logViscosityValuesC_0= np.log10(phiVsMinViscosityC[:,1])
    logPackingFractionC_0 = np.log10(1-phiVsMinViscosityC[:,0]/phi0C)
    logViscosityValuesG_0= np.log10(phiVsMinViscosityG[:,1])
    logPackingFractionG_0 = np.log10(1-phiVsMinViscosityG[:,0]/phi0G)

    logViscosityValuesC_M= np.log10(phiVsMaxViscosityC[:,1])
    logPackingFractionC_M = np.log10(1-phiVsMaxViscosityC[:,0]/phiMC)
    logViscosityValuesG_M= np.log10(phiVsMaxViscosityG[:,1])
    logPackingFractionG_M = np.log10(1-phiVsMaxViscosityG[:,0]/phiMG)



    pyplot.plot(logPackingFractionC_0,logViscosityValuesC_0,'o',label="Frictionless Cornstarch")
    pyplot.plot(logPackingFractionG_0,logViscosityValuesG_0,'o',label="Frictionless Cornstarch/GdCl")
    pyplot.plot(logPackingFractionC_M,logViscosityValuesC_M,'o',label="Frictional Cornstarch")
    pyplot.plot(logPackingFractionG_M,logViscosityValuesG_M,'o',label="Frictional Cornstarch/GdCl")


    pyplot.legend()
    pyplot.xlabel(r"$\log(1-\frac{\phi}{\phi_J})$")
    pyplot.ylabel(r"$\log(\eta)$")



def sideBySideCurvePlot(topDir1,topDir2,topDir3,outputDir,rescaledViscosity=False,allPlots=False,evenCurves="none"):

    #get list of all of the subdirectories in topDir1 and topDir2
    dirsIn1 = next(os.walk(topDir1))[1]
    dirsIn2 = next(os.walk(topDir2))[1]
    dirsIn3 = next(os.walk(topDir3))[1]
    #They need to be sorted so we have to convert them from strings to floats and then back
    dirsIn1 = np.sort([float(i) for i in dirsIn1])
    dirsIn2 = np.sort([float(i) for i in dirsIn2])
    dirsIn3 = np.sort([float(i) for i in dirsIn3])
    #Back to strings!
    dirsIn1 = [str(i) for i in dirsIn1]
    dirsIn2 = [str(i) for i in dirsIn2]
    dirsIn3 = [str(i) for i in dirsIn3]

    pyplot.figure(figsize=(14.0, 5.0))


    markersForPlots = ["o","x","D","h","+","P"]

    for i in range(0,len(dirsIn1)):

        if allPlots == True:
            if evenCurves == "even":
                if i%2!=0:
                    continue
            if evenCurves == "odd":
                if i%2==0:
                    continue





        if rescaledViscosity == True:
            dir1Data = np.load(os.path.join(topDir1,dirsIn1[i], r"averagedDataVR.npy"))
            dir1Errors = np.load(os.path.join(topDir1,dirsIn1[i], r"errorsDataVR.npy"))
            dir2Data = np.load(os.path.join(topDir2,dirsIn2[i], r"averagedDataVR.npy"))
            dir2Errors = np.load(os.path.join(topDir2,dirsIn2[i], r"errorsDataVR.npy"))
            dir3Data = np.load(os.path.join(topDir3,dirsIn3[i], r"averagedDataVR.npy"))
            dir3Errors = np.load(os.path.join(topDir3,dirsIn3[i], r"errorsDataVR.npy"))
        else:
            dir1Data = np.load(os.path.join(topDir1,dirsIn1[i], r"averagedData.npy"))
            dir1Errors = np.load(os.path.join(topDir1,dirsIn1[i], r"errorsData.npy"))
            dir2Data = np.load(os.path.join(topDir2,dirsIn2[i], r"averagedData.npy"))
            dir2Errors = np.load(os.path.join(topDir2,dirsIn2[i], r"errorsData.npy"))
            dir3Data = np.load(os.path.join(topDir3,dirsIn3[i], r"averagedData.npy"))
            dir3Errors = np.load(os.path.join(topDir3,dirsIn3[i], r"errorsData.npy"))


        pyplot.errorbar(dir1Data[:,4],dir1Data[:,2],yerr=dir1Errors[:,2],label=dirsIn1[i]+"C",marker=markersForPlots[i],color="black", markersize=10)
        pyplot.errorbar(dir2Data[:,4],dir2Data[:,2],yerr=dir2Errors[:,2],label=dirsIn2[i]+"GdCl",marker=markersForPlots[i],color="red", markersize=10)
        pyplot.errorbar(dir3Data[:,4],dir3Data[:,2],yerr=dir3Errors[:,2],label=dirsIn3[i]+"NH4",marker=markersForPlots[i],color="blue", markersize=10)

        figName = str(round(float(dirsIn1[i])))
        pyplot.yscale('log')
        pyplot.xscale('log')
        pyplot.xlabel(r"Shear Stress [Pa]")


        if rescaledViscosity == True:
            pyplot.ylabel(r"Rescaled Viscosity $\frac{\eta}{\eta_0}$")
        else:
            pyplot.ylabel(r"Viscosity $\eta$ [Pa s]")

        pyplot.legend()


        if allPlots == False:
            if rescaledViscosity == True:
                pyplot.savefig(os.path.join(outputDir,figName+"rescaled"))
            else:
                pyplot.savefig(os.path.join(outputDir,figName))
            pyplot.clf()
        else:
            pyplot.legend(loc='center left', bbox_to_anchor=(1, 0.5))


    if allPlots == True:
        pyplot.savefig(os.path.join(outputDir,"all"+evenCurves+"plots"))



def shearJammingPhaseDiagram(phi0,phiM,tauStar,alpha):

* Isso gera um gráfico do diagrama de fase de interferência de cisalhamento usando o modelo Wyart-Cates. Este modelo é descrito no papel encontrado aqui:
* https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.112.098302
    
* Este modelo dá a taxa de cisalhamento em função da densidade de sua suspensão e em função da tensão de cisalhamento:
* gammaDot = (1/eta_0)*tau*(1+phi/(phiM+(phi0-phiM)*e^(-tau/tauStar)))^alpha
* onde gammaDot é a taxa de cisalhamento, eta_0 é a viscosidade do solvente de suspensão tau é a tensão de cisalhamento, phi é a densidade, phi0 é a densidade de interferência sem atrito, phiM é a densidade de interferência por fricção e alpha é um parâmetro livre no modelo .
* Estes são os valores y ou as tensões usadas para cada curva

In [ ]:
    stressValues = np.linspace(10**-5,10**5,100000)

* Encontre a linha DST em função dos valores de estresse definidos acima. Esta função retorna frações de empacotamento em função da tensão de cisalhamento

In [ ]:
    def DSTLine(tau):
        return (np.exp(-tau/tauStar)*((phi0-phiM)+phiM*np.exp(tau/tauStar))**2)/(phiM*(np.exp(tau/tauStar)-1-alpha*tau/tauStar)+phi0*(1+alpha*tau/tauStar))

* Encontre a linha Shear Jamming em função dos valores de tensão definidos acima.
* Esta função retorna frações de empacotamento em função da tensão de cisalhamento

In [ ]:
    def shearJammingLine(tau):
        return phiM+(phi0-phiM)*np.exp(-tau/tauStar)

* Estes são os valores x, ou as densidades que definem a linha DST e a linha SJ

In [ ]:
    phiValuesDST = DSTLine(stressValues)
    phiValuesSJ = shearJammingLine(stressValues)

NameError: ignored

* Isso gera a linha vertical que define o bloqueio sem atrito

In [ ]:
   phiValuesJamming = phi0*np.ones(100000)

* Esta é outra linha vertical que não é vista no gráfico, mas atua como limite para a função fill_between

In [ ]:
    phiValuesBeyondJamming = phiValuesJamming+0.06

* Trace todas as quatro curvas, a curva DST, a curva SJ e as duas linhas verticais

In [ ]:
    pyplot.semilogy(phiValuesDST,stressValues,phiValuesSJ,stressValues,phiValuesJamming,stressValues,phiValuesBeyondJamming,stressValues,label = ['DST', 'SJ','J'],color="black")

* Isso sombreia a região entre DST e SJ verde (Esta é a região DST)

In [ ]:
   pyplot.fill_betweenx(stressValues,phiValuesDST,phiValuesSJ,where=phiValuesDST <= phiValuesSJ, facecolor='lightblue', interpolate=True)

* Isso sombreia a região entre SJ e interferência sem fricção azul (Esta é a região SJ)

In [ ]:
    pyplot.fill_betweenx(stressValues,phiValuesSJ,phiValuesJamming,where=phiValuesSJ <= phiValuesJamming, facecolor='lightgreen', interpolate=True)

* Isso sombreia a região entre o bloqueio sem atrito e a linha vertical fora da vista cinza (esta é a região de bloqueio sem atrito)

In [ ]:
    pyplot.fill_betweenx(stressValues,phiValuesJamming,phiValuesBeyondJamming,where=phiValuesJamming <= phiValuesBeyondJamming, facecolor='olive', interpolate=True)


    pyplot.ylim(10**-1, 1000)
    pyplot.xlim(phiM-.1,phi0+.05)

    pyplot.xlabel(r"Packing Fraction $\phi$")
    pyplot.ylabel(r"Shear Stress $ \tau $ [Pa]")
    pyplot.title("Shear Jamming Phase Diagram")






def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]



def shearJammingPhaseDiagramTwoSystems(phi01,phiM1,tauStar1,alpha1,phi02,phiM2,tauStar2,alpha2,labels,DST=False,SJ=False):

* Isso gera um gráfico do diagrama de fase de interferência de cisalhamento usando o modelo Wyart-Cates. Este modelo é descrito no papel encontrado aqui:
    * https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.112.098302
    
* Este modelo dá a taxa de cisalhamento em função da densidade de sua suspensão e em função da tensão de cisalhamento:
* gammaDot = (1/eta_0)*tau*(1+phi/(phiM+(phi0-phiM)*e^(-tau/tauStar)))^alpha
  * onde gammaDot é a taxa de cisalhamento, eta_0 é a viscosidade do solvente de suspensão tau é a tensão de cisalhamento, phi é a densidade, phi0 é a densidade de interferência sem atrito, phiM é a densidade de interferência por fricção e alpha é um parâmetro livre no modelo .
    
   * Estes são os valores y ou as tensões usadas para cada curva

In [ ]:
   stressValues = np.linspace(10**-5,10**5,100000)

* Encontre a linha DST em função dos valores de estresse definidos acima. Esta função retorna frações de empacotamento em função da tensão de cisalhamento

In [ ]:
    def DSTLine(tau,tauStar,phi0,phiM,alpha):
        return (np.exp(-tau/tauStar)*((phi0-phiM)+phiM*np.exp(tau/tauStar))**2)/(phiM*(np.exp(tau/tauStar)-1-alpha*tau/tauStar)+phi0*(1+alpha*tau/tauStar))

* Encontre a linha Shear Jamming em função dos valores de tensão definidos acima. Esta função retorna frações de empacotamento em função da tensão de cisalhamento

In [ ]:
    def shearJammingLine(tau,tauStar,phi0,phiM,alpha):
        return phiM+(phi0-phiM)*np.exp(-tau/tauStar)

* Estes são os valores x, ou as densidades que definem a linha DST e a linha SJ para o sistema 1

In [ ]:
    phiValuesDST1 = DSTLine(stressValues,tauStar1,phi01,phiM1,alpha1)
    phiValuesSJ1 = shearJammingLine(stressValues,tauStar1,phi01,phiM1,alpha1)

* Estes são os valores x, ou as densidades que definem a linha DST e a linha SJ para o sistema 2.

In [ ]:
    phiValuesDST2 = DSTLine(stressValues,tauStar2,phi02,phiM2,alpha2)
    phiValuesSJ2 = shearJammingLine(stressValues,tauStar2,phi02,phiM2,alpha2)

* Isso gera a linha vertical que define o bloqueio sem atrito

In [ ]:
   phiValuesJamming1 = phi01*np.ones(100000)
    phiValuesJamming2 = phi02*np.ones(100000)



    if DST ==True:
        pyplot.semilogy(phiValuesDST1,stressValues,label = 'DST-'+ labels[0],color="black")

* Plote todas as quatro curvas, a curva DST, a curva SJ e as duas linhas verticais para o primeiro sistema.


In [ ]:
        pyplot.semilogy(phiValuesDST2,stressValues,label = 'DST-'+ labels[1],color="red")
        pyplot.fill_betweenx(stressValues,phiValuesDST1,phiValuesSJ1,where=phiValuesDST1 <= phiValuesSJ1, facecolor='lightblue', interpolate=True)
        pyplot.fill_betweenx(stressValues,phiValuesDST2,phiValuesSJ2,where=phiValuesDST2 <= phiValuesSJ2, facecolor='lightgreen', interpolate=True)
        pyplot.semilogy(phiValuesSJ1,stressValues,label = 'SJ-'+ labels[0],color="black")
        pyplot.semilogy(phiValuesSJ2,stressValues,label = 'SJ-'+ labels[1],color="red")
    if SJ == True:
        pyplot.semilogy(phiValuesSJ1,stressValues,label = 'SJ-'+ labels[0],color="black")
        pyplot.semilogy(phiValuesSJ2,stressValues,label = 'SJ-'+ labels[1],color="red")
        pyplot.semilogy(phiValuesJamming1,stressValues,label = 'J-'+ labels[0],color="black")
        pyplot.semilogy(phiValuesJamming2,stressValues,label = 'J-'+ labels[1],color="red")
        pyplot.fill_betweenx(stressValues,phiValuesSJ1,phiValuesJamming1,where=phiValuesSJ1 <= phiValuesJamming1, facecolor='lightgreen', interpolate=True)
        pyplot.fill_betweenx(stressValues,phiValuesSJ2,phiValuesJamming2,where=phiValuesSJ2 <= phiValuesJamming2, facecolor='lightgreen', interpolate=True)



    pyplot.legend()


    phiM = min(phiM1,phiM2)
    phi0 = max(phi01,phi02)

    pyplot.ylim(10**-1, 1000)
    pyplot.xlim(phiM-.05,phi0+.05)

    pyplot.xlabel(r"Packing Fraction $\phi$")
    pyplot.ylabel(r"Shear Stress $ \tau $ [Pa]")
    5
    if DST==True and SJ ==True:
        pyplot.title("Shear Jamming Phase Diagram")

    if DST==True and SJ==False:
        pyplot.title("Discontinuous Shear Thickening Regions")

    if DST==False and SJ==True:
        pyplot.title("Shear Jamming Regions")




def WCModelComparison(topDir,outputDir,phi0,phiM,tauStar,alpha,beta=False):



    listOfSubDirs = next(os.walk(topDir))[1]


    stressValues = np.linspace(.0001,1e3,10000)

    if beta==False:
        def rescaledNewtonianViscosity(stress,phi):
            return (1-phi/(phiM+(phi0-phiM)*np.exp(-stress/tauStar)))**(-alpha)
    else:
        def rescaledNewtonianViscosity(stress,phi):
            return (1-phi/(phi0+(phiM-phi0)*np.exp((-tauStar/stress))**beta))**(-alpha)

    for packingFraction in listOfSubDirs:
        loadInAveragedData = np.load(os.path.join(topDir, packingFraction , r"averagedDataVR.npy" ))
        loadInErrors = np.load(os.path.join(topDir, packingFraction , r"errorsDataVR.npy" ))

        phiValues = (float(packingFraction)/100)*np.ones(10000)

* Plotar a previsão de Wyart-Cates

In [ ]:
       pyplot.plot(stressValues,rescaledNewtonianViscosity(stressValues,phiValues))

* Traçar dados reais

In [ ]:
        pyplot.errorbar(loadInAveragedData[:,4],loadInAveragedData[:,2],yerr=loadInErrors[:,2],label=packingFraction,marker='o',markersize=5)

        pyplot.title(packingFraction)
        pyplot.xlabel("Shear Stress [Pa]")
        pyplot.ylabel("Rescaled Viscosity")
        pyplot.yscale('log')
        pyplot.xscale('log')
        pyplot.ylim(1,1e3)
        pyplot.xlim(1e-1,1e3)
        pyplot.savefig(os.path.join(outputDir,str(float(packingFraction))+".png"))
        pyplot.clf()


def WCModelComparisonTwoSystems(topDir1,topDir2,outputDir,phi01,phiM1,tauStar1,alpha1,phi02,phiM2,tauStar2,alpha2,phi01E,phi02E,phiM1E,phiM2E,tauStar1E,tauStar2E,alpha1E,alpha2E,stretchedExponential=False,beta1=False,beta2=False):



    dirsIn1 = next(os.walk(topDir1))[1]
    dirsIn2 = next(os.walk(topDir2))[1]

* Eles precisam ser classificados, então temos que convertê-los de strings para floats e depois voltar

In [ ]:
    dirsIn1 = np.sort([float(i) for i in dirsIn1])
    dirsIn2 = np.sort([float(i) for i in dirsIn2])

* De volta às strings!

In [ ]:
    dirsIn1 = [str(i) for i in dirsIn1]
    dirsIn2 = [str(i) for i in dirsIn2]


    stressValues = np.linspace(.0001,1e3,10000)

    if stretchedExponential == False:
        def rescaledNewtonianViscosity1(stress,phi):
            return (1-phi/(phiM1+(phi01-phiM1)*np.exp(-stress/tauStar1)))**(-alpha1)

        def rescaledNewtonianViscosity2(stress,phi):
            return (1-phi/(phiM2+(phi02-phiM2)*np.exp(-stress/tauStar2)))**(-alpha2)
    else:
        def rescaledNewtonianViscosity1(stress,phi):
            return (1-phi/(phi01+(phiM1-phi01)*np.exp((-stress/tauStar1)**beta1)))**(-alpha1)

        def rescaledNewtonianViscosity2(stress,phi):
            return (1-phi/(phi02+(phiM2-phi02)*np.exp((-stress/tauStar2)**beta2)))**(-alpha2)





    for i in range(0,len(dirsIn1)):

        loadInAveragedData1 = np.load(os.path.join(topDir1,dirsIn1[i], r"averagedDataVR.npy"))
        loadInErrors1 = np.load(os.path.join(topDir1,dirsIn1[i], r"errorsDataVR.npy"))

        loadInAveragedData2 = np.load(os.path.join(topDir2,dirsIn2[i], r"averagedDataVR.npy"))
        loadInErrors2 = np.load(os.path.join(topDir2,dirsIn2[i], r"errorsDataVR.npy"))

        phiValues1 = (float(dirsIn1[i])/100)*np.ones(10000)
        phiValues2 = (float(dirsIn2[i])/100)*np.ones(10000)

* Plote a previsão de Wyart-Cates para 1

In [ ]:
        pyplot.plot(stressValues,rescaledNewtonianViscosity1(stressValues,phiValues1),label="Cornstarch Fit")

        if stretchedExponential ==False:
            yMinE1 = (1-(float(dirsIn1[i])/100)/((phiM1-phiM1E)+((phi01-phi01E)-(phiM1-phiM1E))*np.exp(-stressValues/(tauStar1-tauStar1E))))**(-(alpha1-alpha1E))
            yMaxE1 = (1-(float(dirsIn1[i])/100)/((phiM1+phiM1E)+((phi01+phi01E)-(phiM1+phiM1E))*np.exp(-stressValues/(tauStar1+tauStar1E))))**(-(alpha1+alpha1E))



        yMinE1 = np.squeeze(yMinE1)


        yMaxE1 = np.squeeze(yMaxE1)


        pyplot.fill_between(stressValues,yMinE1,yMaxE1,yMinE1>=yMaxE1)

* Plotar a previsão de Wyart-Cates para 2

In [ ]:
        pyplot.plot(stressValues,rescaledNewtonianViscosity2(stressValues,phiValues2),label="GdCl Fit")
        if stretchedExponential==False:
            yMinE2 = (1-(float(dirsIn2[i])/100)/((phiM2-phiM2E)+((phi02-phi02E)-(phiM2-phiM2E))*np.exp(-stressValues/(tauStar2-tauStar2E))))**(-(alpha2-alpha2E))
            yMaxE2 = (1-(float(dirsIn2[i])/100)/((phiM2+phiM2E)+((phi02+phi02E)-(phiM2+phiM2E))*np.exp(-stressValues/(tauStar2+tauStar2E))))**(-(alpha2+alpha2E))



        yMinE2 = np.squeeze(yMinE2)
        yMaxE2 = np.squeeze(yMaxE2)
        pyplot.fill_between(stressValues,yMinE2,yMaxE2,yMinE2>=yMaxE2)

        pyplot.errorbar(loadInAveragedData1[:,4],loadInAveragedData1[:,2],yerr=loadInErrors1[:,2],label=dirsIn1[i]+"C",marker='o')
        pyplot.errorbar(loadInAveragedData2[:,4],loadInAveragedData2[:,2],yerr=loadInErrors2[:,2],label=dirsIn2[i]+"GdCl",marker="D")

        figName = str(round(float(dirsIn1[i])))

        pyplot.title(figName)
        pyplot.xlabel("Shear Stress [Pa]")
        pyplot.ylabel("Rescaled Viscosity")
        pyplot.yscale('log')
        pyplot.xscale('log')
        pyplot.ylim(1,5e3)
        pyplot.xlim(1e-1,1e3)
        pyplot.legend()
        pyplot.savefig(os.path.join(outputDir,str(float(figName))+".png"))
        pyplot.clf()